In [10]:
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached Brotli-1.1.0-cp312-cp312-macosx_10_13_universal2.whl.metadata (5.5 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.12.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached pandas-2.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  

In [11]:
from langchain_openai import ChatOpenAI  # For creating the LLM model instances
from langchain.prompts import (
    SystemMessagePromptTemplate, # For creating system prompt templates
    HumanMessagePromptTemplate, # For creating 'user' prompt templates
    ChatPromptTemplate, # For creating consolidated prompt template of SYSTEM + USER
    PromptTemplate # For creating a general prompt template
)
from pydantic import BaseModel, Field # For creating data models for structured output
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper # For generating images
from langchain_core.runnables import RunnableLambda # For using functions as runnable chain blocks
from langchain.prompts import FewShotChatMessagePromptTemplate # For Few Shot Prompting
from IPython.display import display, Markdown  # For markdown formatting
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser

In [12]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

In [13]:
llm = ChatOpenAI(temperature=1.0)

In [14]:
### Output parsers
class ExtractedInfo(BaseModel):
  name: str = Field(description="This contains the full name of the applicant specified on their resume (Or only first name if full name not given)")
  email: str = Field(description="The email address of the applicant specified on their resume")
  skills: list[str] = Field(description="This will be a list of different skills that can be inferred from the applicant's resume + any additional skills specifically mentioned on their resume")

output_parser = PydanticOutputParser(pydantic_object=ExtractedInfo)
format_instructions = output_parser.get_format_instructions()

### Prompt templates
prompt = PromptTemplate(
  template="""
  You are an assistant to the Hiring Department of an Artificial Intelligence Company. 
  You are given the raw text of an applicant's resume below.
  {resumeText}

  You must extract key details of: Name, Email, and Technical Skills from the resume provided
  where the skills corresponds to not only any specifically mentioned skills on the
  resume text, but also what you can infer from the resume.

  {format_instructions}
  """,
  input_variables=['resumeText'],
  partial_variables={'format_instructions': format_instructions}
)

In [27]:
from langchain.document_loaders import PyPDFLoader

### Build the chain executable function
chain = prompt | llm

def run_chain(file):
  loader = PyPDFLoader(file)
  pages = loader.load_and_split()
  resumeText = ""
  for page in pages:
    resumeText+=page.page_content

  result = chain.invoke(resumeText)
  result = output_parser.parse(result.content) # This will be an ExtractedInfo object
  return f"""
  Name of applicant: {result.name}
  Email address of applicant: {result.email}
  Skills of applicant: {", ".join(result.skills)}
  """

In [28]:
import gradio as gr

def process_resume(files, number, text):
  for file in files:
    try:
      result = run_chain(file)
      return result
    except Exception as e:
      return f"Error reading file: {e}"
  return f"The number is {number} and the text is {text}"

demo = gr.Interface(
  fn = process_resume,
  inputs = [
    gr.File(label="Select Resume", file_count="multiple"),
  ],
  outputs = gr.Textbox(label = "output")
)

demo.launch()

/Users/gobus/Desktop/main/COURSES/LangChain for LLM App Dev/.venv/lib/python3.12/site-packages/gradio/utils.py:1052: UserWarning: Expected 3 arguments for function <function process_resume at 0x16e1f9ee0>, received 1.
  warnings.warn(
/Users/gobus/Desktop/main/COURSES/LangChain for LLM App Dev/.venv/lib/python3.12/site-packages/gradio/utils.py:1056: UserWarning: Expected at least 3 arguments for function <function process_resume at 0x16e1f9ee0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


/Users/gobus/Desktop/main/COURSES/LangChain for LLM App Dev/.venv/lib/python3.12/site-packages/gradio/helpers.py:1057: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


In [18]:
!pip install pypdf

  Using cached pypdf-6.0.0-py3-none-any.whl.metadata (7.1 kB)
Using cached pypdf-6.0.0-py3-none-any.whl (310 kB)
